In [1]:
import pandas as pd
import numpy as np
import os
from numpy import exp
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import preprocessing
from math import pi
from numpy.matlib import repmat
from copy import deepcopy
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler 
import pywt
from scipy.fftpack import fft
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler
from sklearn.kernel_approximation import RBFSampler, Nystroem
from sklearn.decomposition import PCA, KernelPCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
import warnings
import logging
import smote_variants as sv
import imblearn
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
logging.getLogger('smote_variants').setLevel(logging.WARNING)
logging.getLogger('xgboost').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, message=".*use_label_encoder.*")
warnings.filterwarnings("ignore", category=DeprecationWarning, message=".*ndim > 0 to a scalar.*")
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.neural_network')

In [2]:
# در متد زیر فاصله هر سطر (نمونه ماترسی) از نمونه های دیگر حساب می شود متد زیر به صورت کلی و با ماتریس های چند بعدی کار می کند
def L2_dist(A,B):
    x = A
    xsup = B
    ps = np.dot(A, B.T)
    nps,pps = ps.shape
    normx = np.sum(x**2,axis =1)
    normxsup = np.sum(xsup**2, axis = 1)
    ps = -2 * ps + normx[:, np.newaxis] + normxsup[np.newaxis, :]  # درست سازی ابعاد برای جمع زدن
    E = np.sqrt(np.maximum(ps, 0))
    
    return E


def KNNenemyPoint(mindata,majdata):
    k_nNum = 1
    NT = mindata.shape[0]
    NTNT = majdata.shape[0]


    # for minority :
    
    minENNdist=np.ones((NT,k_nNum))*np.inf
    minenemykneigor = np.ones((NT,k_nNum))*np.inf

    # for majority
    majENNdist= np.ones((NTNT,k_nNum))*np.inf
    majenemykneigor = np.ones((NTNT,k_nNum))*np.inf

    # calculate nearest enemy from minoriy samples

    pd = L2_dist(mindata,majdata) # distance matrix betwen minority and majority
    for i in range(0,NT):
        
        d =deepcopy(pd[i,:])    # read row of distance matrix in each repeat
        min_id = []
       
        for k in range(0,k_nNum):
            id = np.argmin(d)
            d[id] = np.inf
            min_id.append(id)
        
        minENNdist[i, :] = pd[i, min_id]  #  فاصله نزدیک ترین نمونه به سطر مربوطه در اینجا ذخیره می شود
        minenemykneigor[i, :] = min_id # ایدی در اینجا ذخیره می شود
   
 
    
    # calculate nearest enemy from majority samples
    pd = pd.T
    majmindistmax = pd

    for i in range(0,NTNT):
        
        d =deepcopy(pd[i,:])    # read row of  distance transpose matrix in each repeat
        min_id = []
        for k in range(0,k_nNum):
            id = np.argmin(d)
            d[id] = np.inf
            min_id.append(id)

        majENNdist[i,:] = pd[i,min_id]
        majenemykneigor[i,:] = min_id


    return minENNdist , minenemykneigor , majENNdist , majenemykneigor , majmindistmax
        
    
def calradious(i,ilabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor):   # برچسب هارا جداگانه مدیریت می کنیم
    # i index of sample in dataset
    # ilabel label of sample i 
    IR = majENNdist.shape[0] / minENNdist.shape[0]
    
    tha = 1 / (1 + np.exp(-(IR - 1)))  # Sigmoid-based threshold

    if ilabel == minlabel:
        nej =  int((minenemykneigor[i])[0])
        dij = minENNdist[i]
        jlabel = majlabel
        if i == majenemykneigor[nej]:
            dradius = dij * tha
            return dradius
        else:
            dt = calradious(nej, jlabel, majlabel, minlabel,minENNdist, minenemykneigor, majENNdist, majenemykneigor)

            dradius = dij - dt
            return dradius

    else:
        nej = int((majenemykneigor[i])[0])
        dij = majENNdist[i]
        jlabel = minlabel

        if i == minenemykneigor[nej]:
            dradius =  dij * (1 - tha)
            return dradius
        else:
            dt = calradious(nej, jlabel, majlabel, minlabel, minENNdist, minenemykneigor, majENNdist, majenemykneigor)
            dradius = dij - dt
            return dradius
import numpy as np

def normalizemeanstd(xapp, xtest=None, meanx=None, stdx=None):
    """
    Normalize inputs so that they have mean 0 and standard deviation 1.

    Parameters:
    - xapp: Training data (numpy array).
    - xtest: Test data (numpy array, optional).
    - meanx: Precomputed mean values (optional).
    - stdx: Precomputed standard deviation values (optional).

    Returns:
    - xapp: Normalized training data.
    - xtest: Normalized test data (if provided).
    - meanxapp: Mean values used for normalization.
    - stdxapp: Standard deviation values used for normalization.
    """

    tol = 1e-5  # Tolerance for near-zero standard deviation

    # Compute mean and standard deviation if not provided
    if meanx is None or stdx is None:
        meanxapp = np.mean(xapp, axis=0)  # Compute column-wise mean
        stdxapp = np.std(xapp, axis=0)    # Compute column-wise standard deviation
    else:
        meanxapp = meanx
        stdxapp = stdx

    # Handle near-zero standard deviations to avoid division errors
    stdxapp[np.abs(stdxapp) < tol] = 1  

    # Normalize training data
    xapp = (xapp - meanxapp) / stdxapp

    # Normalize test data if provided
    if xtest is not None and len(xtest) > 0:
        xtest = (xtest - meanxapp) / stdxapp
    else:
        xtest = None

    return xapp, xtest, meanxapp, stdxapp
#در کد زیر ابتدا شعاع ها را از کوچک به بزرگ مرتب سازی می کنیم سپس  بزرگترین کره ی فعلی را انتخاب می کنیم و سپس فاصله ی نمونه های بعدی را با شعاع این کره مقایسه می کنیم و سپس بررسی می کنیم که نمونه های بعدی متعلق به کدام کره اند

def  unionSphere(data,dataradious):


    # Step 1: Calculate the pairwise Euclidean distance matrix
    pddist = L2_dist(data,data)

    # Step 2: Sort dataradious by the last column in descending order
    sortid = np.argsort(dataradious[:, -1])[::-1]  # Get indices for descending order
    sortdataradious = dataradious[sortid]  # Sort dataradious by radius (last column)


    # Initialize the flag array, marking which points have been processed
    flag = np.zeros(data.shape[0], dtype=int)

    # Step 3: Loop through each point and process it
    for i in range(sortdataradious.shape[0]):
        if flag[i] == 0:  # If the point has not been processed yet
            flag[i] = sortdataradious[i, 0]  # Mark it with its own identifier (from dataradious)

            # Get the indices of the remaining points that haven't been processed
            leftindex = np.arange(i + 1, sortdataradious.shape[0])
            leftindex = leftindex[flag[leftindex] == 0]  # Points that haven't been processed

            # Get the identifiers of the remaining unprocessed points
            leftunhandledid = (sortdataradious[leftindex, 0]).astype(int)


        
            # Calculate the distances from the current point to the remaining unprocessed points
            leftdist = pddist[int(sortdataradious[i, 0]), leftunhandledid] # فاصله ی نمونه های بررسی نشده تا مرکز انتخابی 

            # Find the indices of points that satisfy the distance condition
            includedset = np.where(leftdist <= sortdataradious[i, -1])[0] # befoe minus -1 it was 3 (last element of dataradious matrix )

            # If there are any included points, mark them with the current point's identifier
            if includedset.size > 0:
                flag[leftindex[includedset]] = flag[i]

    # Step 4: Create the "belongto" array to mark which group each point belongs to
    belongto = np.zeros(dataradious.shape[0], dtype=int)
    belongto[sortdataradious[:, 0].astype(int)] = flag

    # Update the dataradious with the new 'belongto' column
    dataradious = np.hstack([dataradious, belongto[:, np.newaxis]])  #****************************** table of information ****************************#

    
    
    
    #sorted_dataradious = dataradious[dataradious[:, -2].argsort()]
    

    return dataradious, pddist


def gnerandsphere(center, d, N, r):
    """
    Generates N random points within a d-dimensional sphere centered at `center` with radius `r`.

    Parameters:
    center (array-like): Coordinates of the sphere center.
    d (int): Dimension of the sphere.
    N (int): Number of points to generate.
    r (float): Radius of the sphere.

    Returns:
    np.ndarray: Nxd array of generated points.
    """
    # Generate random directions (unit vectors)
    random_directions = np.random.randn(N, d)
    random_directions /= np.linalg.norm(random_directions, axis=1, keepdims=True)  # Normalize to unit vectors

    # Generate random radii
    random_radii = r * np.random.rand(N, 1)

    # Scale unit vectors by random radii
    Y = random_directions * random_radii

    # Shift by the center
    Y = np.array(center) + Y

    return Y

# train test split

In [4]:
def getTrainAndTest(data_set_name, n_splits=5 , majlabel = +1 , minlabel = -1):
    # Load data
    data_all = np.loadtxt(data_set_name)
    
    # Shuffle dataset after all folds are completed
    np.random.shuffle(data_all)
    
    # Dimension
    dim = data_all[:, :-1].shape[1]
    
    # Normalize features
    X_normalized, _, meanxapp, stdxapp = normalizemeanstd(data_all[:, :-1])
    
    # Extract labels
    y = data_all[:, -1]
    
    # Initialize Stratified K-Fold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=None)
    
    # Iterate through folds
    for train_index, test_index in skf.split(X_normalized, y):
        X_train, X_test = X_normalized[train_index], X_normalized[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Convert y_train to a column vector for concatenation
        y_train = y_train.reshape(-1, 1)
        
        # Append labels to data_allmajority and data_allminority
        data_allmajority = np.hstack((X_train[y_train.flatten() == majlabel], y_train[y_train.flatten() == majlabel]))
        data_allminority = np.hstack((X_train[y_train.flatten() == minlabel], y_train[y_train.flatten() == minlabel]))
        
        # Calculate imbalance ratio
        IMratio = data_allmajority.shape[0] / data_allminority.shape[0] if data_allminority.shape[0] > 0 else np.inf
        
        yield dim, X_train, X_test, y_train, y_test, data_allmajority, data_allminority, IMratio



def getDataAlgorithmCylinderSADCO(data_allminority,data_allmajority,generatedX , new_generatedX , minlabel = -1 ):


    
    column_of_neg_ones = minlabel * np.ones((generatedX.shape[0], 1))

    generatedX = np.hstack((generatedX, column_of_neg_ones))
    
    column_of_neg_ones = minlabel* np.ones((new_generatedX.shape[0], 1))
    
    new_generatedX =np.hstack((new_generatedX, column_of_neg_ones))



    data = np.concatenate((data_allminority,generatedX,new_generatedX,data_allmajority), axis=0)



    X_train = data[:,:-1]
    y_train = data[:,-1]

    #print("Train length is :  " , len(X_train))
   

    #print("Number of majority class in train is : ",np.sum(y_train[:]==1))
    #print("Number of minority class in train is : ",np.sum(y_train[:]==-1))
    

    return X_train, y_train


def getDataForSADCO(data_allminority,data_allmajority,generatedX , minlabel = -1):
    column_of_neg_ones = minlabel * np.ones((generatedX.shape[0], 1))

    generatedX = np.hstack((generatedX, column_of_neg_ones))
    
    

    data = np.concatenate((data_allminority,generatedX,data_allmajority), axis=0)

    X_train = data[:,:-1]
    y_train = data[:,-1]
    
    #print("Train length is :  " , len(X_train))
   

    #print("Number of majority class in train is : ",np.sum(y_train[:]==1))
    #print("Number of minority class in train is : ",np.sum(y_train[:]==-1))    

    return X_train, y_train
def getDataForSMOTE(X_train , y_train):

    X_train = np.array(X_train)
    y_train = np.array(y_train).flatten()  
    
    oversampler = sv.SMOTE(n_neighbors=5, proportion=1.0, random_state=42)
    X_SMOTE, y_SMOTE = oversampler.fit_resample(X_train, y_train)
    
    return X_SMOTE, y_SMOTE
    
def getDataForSSMOTE(X_train , y_train):

    X_train = np.array(X_train)
    y_train = np.array(y_train).flatten()
    
    oversampler = sv.Safe_Level_SMOTE(n_neighbors=5, proportion=1.0, random_state=42)
    X_SSMOTE, y_SSMOTE = oversampler.fit_resample(X_train, y_train)
    return X_SSMOTE,y_SSMOTE

def getDataForBSMOTE(X_train , y_train):
    X_train = np.array(X_train)
    y_train = np.array(y_train).flatten()
    
    borderline_smote = imblearn.over_sampling.BorderlineSMOTE(k_neighbors=2,random_state=42)
    X_BSMOTE, y_BSMOTE = borderline_smote.fit_resample(X_train, y_train)
    return X_BSMOTE,y_BSMOTE
    #k_neighbors=2 for zoo dataset for others we used default parameters

def getDataForADASYN(X_train , y_train):
    X_train = np.array(X_train)
    y_train = np.array(y_train).flatten()
    
    oversampler = sv.ADASYN(n_neighbors=5, proportion=1.0, random_state=42)
    X_ADASYN, y_ADASYN = oversampler.fit_resample(X_train, y_train)
    return X_ADASYN,y_ADASYN

def getDataForMWMOTE(X_train , y_train):
    X_train = np.array(X_train)
    y_train = np.array(y_train).flatten()
    
    oversampler = sv.MWMOTE(n_neighbors=5, proportion=1.0, random_state=42)
    X_MWMOTE, y_MWMOTE = oversampler.fit_resample(X_train, y_train)
    return X_MWMOTE , y_MWMOTE

def getDataForA_SUWO(X_train , y_train):
    X_train = np.array(X_train)
    y_train = np.array(y_train).flatten()
    
    oversampler = sv.A_SUWO(n_neighbors=5, proportion=1.0, random_state=42,)
    X_ASUWO, y_ASUWO = oversampler.fit_resample(X_train, y_train)
    return X_ASUWO , y_ASUWO
    

def getDataForKMeansSMOTE(X_train , y_train):
    X_train = np.array(X_train)
    y_train = np.array(y_train).flatten()
    
    oversampler = sv.kmeans_SMOTE(k_neighbors=5, proportion=1.0, random_state=42)
    X_KSMOTE, y_KSMOTE = oversampler.fit_resample(X_train, y_train)
    return X_KSMOTE , y_KSMOTE
    
    
def getDataForGSMOTE(X_train , y_train):
    X_train = np.array(X_train)
    y_train = np.array(y_train).flatten()
    
    oversampler = sv.G_SMOTE(k_neighbors=5, proportion=1.0, random_state=42)
    X_GSMOTE, y_GSMOTE = oversampler.fit_resample(X_train, y_train)
    return X_GSMOTE , y_GSMOTE



# train model

In [6]:
def evaluate_models(X_train, y_train, X_test, y_test, feature_expansion="original"):
    # Ensure input data is float32
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # Map original labels: -1 (positive) -> 1, 1 (negative) -> 0
    label_map = {-1: 1, 1: 0}
    y_train = np.array([label_map[y] for y in y_train], dtype=np.int32)
    y_test = np.array([label_map[y] for y in y_test], dtype=np.int32)

    # Define classifiers
    classifiers = {
        "Random Forest": RandomForestClassifier(random_state=42),
        "SVM": SVC(kernel='rbf', probability=True, random_state=42),
        "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42),
    }

    results = {}

    for name, clf in classifiers.items():
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        # Get predicted probabilities for positive class label = 1
        pos_class_index = np.where(clf.classes_ == 1)[0][0]
        y_prob = clf.predict_proba(X_test)[:, pos_class_index]

        auc_score = roc_auc_score(y_test, y_prob)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, pos_label=1)
        g_mean = geometric_mean_score(y_test, y_pred, pos_label=1)

        class_report = classification_report(
            y_test,
            y_pred,
            target_names=['negative', 'positive'],
            output_dict=True,
            zero_division=1
        )

        results[name] = {
            'AUC': auc_score,
            'Accuracy': accuracy,
            'F1-score': f1,
            'G-Mean': g_mean,
            'Classification Report': class_report
        }

    return results



def getResults(models , sum_metrics , results ):
    for model in models :
        sum_metrics[model]['AUC'] += results[model]['AUC']
        sum_metrics[model]['F1-score'] += results[model]['F1-score']
        sum_metrics[model]['G-Mean'] += results[model]['G-Mean']
        sum_metrics[model]['Recall']['-1.0'] += results[model]['Classification Report']['positive']['recall']
        sum_metrics[model]['Recall']['1.0'] += results[model]['Classification Report']['negative']['recall']
        sum_metrics[model]['Precision']['-1.0'] += results[model]['Classification Report']['positive']['precision']
        sum_metrics[model]['Precision']['1.0'] += results[model]['Classification Report']['negative']['precision']
    return sum_metrics



    
def printResults(data_set_name, th, density, degree, keep, th_end, limit, use_enemy, th2 ,density2, th_end2, limit2 , use_enemy2, n_splits=5, times=3):
    # Initialize sums for all metrics
    models = ['Random Forest' , 'XGBoost','SVM' ]
    
    sum_metrics_1 = {model: {'AUC': 0, 'F1-score': 0, 'G-Mean': 0, 'Recall': {'-1.0': 0, '1.0': 0}, 'Precision': {'-1.0': 0, '1.0': 0}} for model in models}
    sum_metrics_2 = {model: {'AUC': 0, 'F1-score': 0, 'G-Mean': 0, 'Recall': {'-1.0': 0, '1.0': 0}, 'Precision': {'-1.0': 0, '1.0': 0}} for model in models}
    sum_metrics_3 = {model: {'AUC': 0, 'F1-score': 0, 'G-Mean': 0, 'Recall': {'-1.0': 0, '1.0': 0}, 'Precision': {'-1.0': 0, '1.0': 0}} for model in models}
    sum_metrics_SMOTE = {model: {'AUC': 0, 'F1-score': 0, 'G-Mean': 0, 'Recall': {'-1.0': 0, '1.0': 0}, 'Precision': {'-1.0': 0, '1.0': 0}} for model in models}
    sum_metrics_SSMOTE ={model: {'AUC': 0, 'F1-score': 0, 'G-Mean': 0, 'Recall': {'-1.0': 0, '1.0': 0}, 'Precision': {'-1.0': 0, '1.0': 0}} for model in models}
    sum_metrics_BSMOTE = {model: {'AUC': 0, 'F1-score': 0, 'G-Mean': 0, 'Recall': {'-1.0': 0, '1.0': 0}, 'Precision': {'-1.0': 0, '1.0': 0}} for model in models}
    sum_metrics_ADASYN ={model: {'AUC': 0, 'F1-score': 0, 'G-Mean': 0, 'Recall': {'-1.0': 0, '1.0': 0}, 'Precision': {'-1.0': 0, '1.0': 0}} for model in models}
    sum_metrics_MWMOTE = {model: {'AUC': 0, 'F1-score': 0, 'G-Mean': 0, 'Recall': {'-1.0': 0, '1.0': 0}, 'Precision': {'-1.0': 0, '1.0': 0}} for model in models}
    sum_metrics_ASUWO ={model: {'AUC': 0, 'F1-score': 0, 'G-Mean': 0, 'Recall': {'-1.0': 0, '1.0': 0}, 'Precision': {'-1.0': 0, '1.0': 0}} for model in models}
    sum_metrics_KMSMOTE ={model: {'AUC': 0, 'F1-score': 0, 'G-Mean': 0, 'Recall': {'-1.0': 0, '1.0': 0}, 'Precision': {'-1.0': 0, '1.0': 0}} for model in models}
    sum_metrics_GSMOTE ={model: {'AUC': 0, 'F1-score': 0, 'G-Mean': 0, 'Recall': {'-1.0': 0, '1.0': 0}, 'Precision': {'-1.0': 0, '1.0': 0}} for model in models}
    

    
    # Define the main directory with dataset name
    dataset_dir = os.path.join("final_experiment_results", data_set_name)
    os.makedirs(dataset_dir, exist_ok=True)
    
    for time in range(times):
        # Create a directory for each run inside the dataset directory
        run_dir = os.path.join(dataset_dir, f"run_{time+1}")
        os.makedirs(run_dir, exist_ok=True)
    
        for fold_idx, fold in enumerate(getTrainAndTest(data_set_name, n_splits=n_splits)):
            # Create a folder for each fold inside the current run directory
            fold_dir = os.path.join(run_dir, f"fold_{fold_idx+1}")
            os.makedirs(fold_dir, exist_ok=True)
    
            # Extract original train and test sets
            dim, X_train_original, X_test_original, y_train_original, y_test_original, data_allmajority, data_allminority, IMratio = fold
    
            # Function to save NumPy arrays as TXT
            def save_as_txt(filename, array):
                np.savetxt(os.path.join(fold_dir, filename), array, fmt="%s")
    
            # Save original train and test datasets as TXT
            save_as_txt("X_train_original.txt", X_train_original)
            save_as_txt("y_train_original.txt", y_train_original)
            save_as_txt("X_test_original.txt", X_test_original)
            save_as_txt("y_test_original.txt", y_test_original)
    
            # Generate synthetic data 
            
            data_allminority, data_allmajority, mindradious, uncoveredpoint, sorted_dataradious, finalmincenters, finalminradius, numofsphere, generatedX = GENERATEWITHSADCO(
                data_set_name, dim, data_allmajority, data_allminority, IMratio, majlabel=+1, minlabel=-1)
    
            finalmincenters1, finalminradius1, new_finalmincenters1, new_finalminradius1, new_generatedX1, from_center1_to_center2_1, from_sample_to_line1 = cylinder1(
                data_allminority, data_allmajority, finalmincenters, finalminradius, generatedX, th=th, density=density, degree=degree, keep=keep, th_end=th_end, limit=limit, use_enemy=use_enemy)


            finalmincenters2, finalminradius2, left_axis2, right_axis2, radius2, new_generatedX2, from_center1_to_center2_2=cylinder2(data_allminority, data_allmajority, finalmincenters, 
                                                                                                finalminradius, generatedX, th = th2, density = density2,  th_end= th_end2, limit=limit2, use_enemy = use_enemy2)
            
            # Get training datasets
            X_train_SADCOC1, y_train_SADCOC1 = getDataAlgorithmCylinderSADCO(data_allminority, data_allmajority, generatedX, new_generatedX1)
            X_train_SADCOC2, y_train_SADCOC2 = getDataAlgorithmCylinderSADCO(data_allminority, data_allmajority, generatedX, new_generatedX2)
            X_train_SADCO, y_train_SADCO = getDataForSADCO(data_allminority, data_allmajority, generatedX)
            X_train_SMOTE , y_train_SMOTE = getDataForSMOTE(X_train_original,y_train_original)
            X_train_SSMOTE , y_train_SSMOTE = getDataForSSMOTE(X_train_original,y_train_original)
            X_train_BSMOTE , y_train_BSMOTE = getDataForBSMOTE(X_train_original,y_train_original)
            X_train_ADASYN , y_train_ADASYN = getDataForADASYN(X_train_original,y_train_original)
            X_train_MWMOTE , y_train_MWMOTE = getDataForMWMOTE(X_train_original,y_train_original)
            X_train_ASUWO , y_train_ASUWO = getDataForA_SUWO(X_train_original , y_train_original)
            X_train_KMSMOTE , y_train_KMSMOTE = getDataForKMeansSMOTE(X_train_original , y_train_original)
            X_train_GSMOTE , y_train_GSMOTE = getDataForGSMOTE(X_train_original , y_train_original)
            
    
            
            save_as_txt("X_train_SADCOC1.txt", X_train_SADCOC1)
            save_as_txt("X_train_SADCOC2.txt", X_train_SADCOC2)
            save_as_txt("X_train_SADCO.txt", X_train_SADCO)
            save_as_txt("X_train_SMOTE.txt", X_train_SMOTE)
            save_as_txt("X_train_SSMOTE.txt", X_train_SSMOTE)
            save_as_txt("X_train_BSMOTE.txt", X_train_BSMOTE)
            save_as_txt("X_train_ADASYN.txt", X_train_ADASYN)
            save_as_txt("X_train_MWMOTE.txt", X_train_MWMOTE)
            save_as_txt("X_train_ASUWO.txt", X_train_ASUWO)
            save_as_txt("X_train_KMSMOTE.txt", X_train_KMSMOTE)
            save_as_txt("X_train_GSMOTE.txt", X_train_GSMOTE)

            save_as_txt("y_train_SADCOC1.txt", y_train_SADCOC1)
            save_as_txt("y_train_SADCOC2.txt", y_train_SADCOC2)
            save_as_txt("y_train_SADCO.txt", y_train_SADCO)
            save_as_txt("y_train_SMOTE.txt", y_train_SMOTE)
            save_as_txt("y_train_SSMOTE.txt", y_train_SSMOTE)
            save_as_txt("y_train_BSMOTE.txt", y_train_BSMOTE)
            save_as_txt("y_train_ADASYN.txt", y_train_ADASYN)
            save_as_txt("y_train_MWMOTE.txt", y_train_MWMOTE)
            save_as_txt("y_train_ASUWO.txt", y_train_ASUWO)
            save_as_txt("y_train_KMSMOTE.txt", y_train_KMSMOTE)
            save_as_txt("y_train_GSMOTE.txt", y_train_GSMOTE)

            
           
    
           
            results1 = evaluate_models(X_train_SADCOC1, y_train_SADCOC1, X_test_original, y_test_original)
            with open(os.path.join(fold_dir, "results_SADCOC1.txt"), "w") as f:
                f.write(str(results1))

            results2 = evaluate_models(X_train_SADCOC2, y_train_SADCOC2, X_test_original, y_test_original)
            with open(os.path.join(fold_dir, "results_SADCOC2.txt"), "w") as f:
                f.write(str(results2))
    

            results3 = evaluate_models(X_train_SADCO, y_train_SADCO, X_test_original, y_test_original)
            with open(os.path.join(fold_dir, "results_SADCO.txt"), "w") as f:
                f.write(str(results3))

            results_SMOTE = evaluate_models(X_train_SMOTE,y_train_SMOTE,X_test_original, y_test_original)
            with open(os.path.join(fold_dir, "results_SMOTE.txt"), "w") as f:
                f.write(str(results_SMOTE))
                
            results_SSMOTE=evaluate_models(X_train_SSMOTE,y_train_SSMOTE,X_test_original, y_test_original)
            with open(os.path.join(fold_dir, "results_SSMOTE.txt"), "w") as f:
                f.write(str(results_SSMOTE))

            
            results_BSMOTE=evaluate_models(X_train_BSMOTE,y_train_BSMOTE,X_test_original, y_test_original)
            with open(os.path.join(fold_dir, "results_BSMOTE.txt"), "w") as f:
                f.write(str(results_BSMOTE))
                
            results_ADASYN=evaluate_models(X_train_ADASYN,y_train_ADASYN,X_test_original, y_test_original)
            with open(os.path.join(fold_dir, "results_ADASYN.txt"), "w") as f:
                f.write(str(results_ADASYN))
                
            results_MWMOTE=evaluate_models(X_train_MWMOTE,y_train_MWMOTE,X_test_original, y_test_original)
            with open(os.path.join(fold_dir, "results_MWMOTE.txt"), "w") as f:
                f.write(str(results_MWMOTE))
                
            results_ASUWO=evaluate_models(X_train_ASUWO,y_train_ASUWO,X_test_original, y_test_original)
            with open(os.path.join(fold_dir, "results_ASUWO.txt"), "w") as f:
                f.write(str(results_ASUWO))
                
            results_KMSMOTE = evaluate_models(X_train_KMSMOTE,y_train_KMSMOTE,X_test_original, y_test_original)
            with open(os.path.join(fold_dir, "results_KMSMOTE.txt"), "w") as f:
                f.write(str(results_KMSMOTE))
                
            results_GSMOTE = evaluate_models(X_train_GSMOTE,y_train_GSMOTE,X_test_original, y_test_original)
            with open(os.path.join(fold_dir, "results_GSMOTE.txt"), "w") as f:
                f.write(str(results_GSMOTE))  
                


        
        

            
        # Aggregate results for each model

            sum_metrics_1= getResults(models,sum_metrics_1,results1)
            sum_metrics_2 =getResults(models,sum_metrics_2,results2)
            sum_metrics_3 =getResults(models,sum_metrics_3,results3)
            
            sum_metrics_SMOTE=getResults(models,sum_metrics_SMOTE,results_SMOTE)
            sum_metrics_SSMOTE=getResults(models, sum_metrics_SSMOTE,results_SSMOTE)
            sum_metrics_BSMOTE=getResults(models,sum_metrics_BSMOTE,results_BSMOTE)
            sum_metrics_ADASYN=getResults(models,sum_metrics_ADASYN,results_ADASYN)
            sum_metrics_MWMOTE=getResults(models,sum_metrics_MWMOTE,results_MWMOTE)
            sum_metrics_ASUWO=getResults(models,sum_metrics_ASUWO,results_ASUWO)
            sum_metrics_KMSMOTE=getResults(models, sum_metrics_KMSMOTE,results_KMSMOTE)
            sum_metrics_GSMOTE=getResults(models,sum_metrics_GSMOTE,results_GSMOTE)
    
    # Calculate and print averages
    
    def print_results(label, metrics,filename,dataset_dir):
        print(f"{label} Results:")
        save_results = {}
        for model in models:
            avg_auc = metrics[model]['AUC'] / (n_splits * times)
            avg_fscore = metrics[model]['F1-score'] / (n_splits * times)
            avg_gmean = metrics[model]['G-Mean'] / (n_splits * times)
            avg_recall_minus1 = metrics[model]['Recall']['-1.0'] / (n_splits * times)
            avg_recall_1 = metrics[model]['Recall']['1.0'] / (n_splits * times)
            avg_precision_minus1 = metrics[model]['Precision']['-1.0'] / (n_splits * times)
            avg_precision_1 = metrics[model]['Precision']['1.0'] / (n_splits * times)
            save_results[model]= {
                                    'avg_auc':avg_auc,
                                    'avg_fscore':avg_fscore,
                                    'avg_gmean':avg_gmean,
                                    'avg_recall_minus1':avg_recall_minus1 ,
                                    'avg_recall_1': avg_recall_1,
                                    'avg_precision_minus1':avg_precision_minus1,
                                    'avg_precision_1':avg_precision_1,

                                    }
        

            print(f"{model}:")
            print(f"  AUC: {avg_auc}")
            print(f"  F1-score: {avg_fscore}")
            print(f"  G-Mean: {avg_gmean}")
            print(f"  Recall Class -1.0: {avg_recall_minus1}")
            print(f"  Recall Class 1.0: {avg_recall_1}")
            print(f"  Precision Class -1.0: {avg_precision_minus1}")
            print(f"  Precision Class 1.0: {avg_precision_1}")
            print()
        with open(os.path.join(dataset_dir, filename), "w") as f:
            f.write(str(save_results))
    
    print_results("SADCOC1", sum_metrics_1 , filename='SADCOC1-AVERAGE' ,dataset_dir = dataset_dir )
    print_results("SADCOC2", sum_metrics_2 , filename = 'SADCOC2-AVERAGE' ,dataset_dir = dataset_dir )
    print_results("SADCO", sum_metrics_3 , filename = 'SADCO-AVERAGE' ,dataset_dir = dataset_dir )

    print_results("SMOTE", sum_metrics_SMOTE , filename = 'SMOTE-AVERAGE' ,dataset_dir = dataset_dir )
    print_results("SSMOTE", sum_metrics_SSMOTE , filename = 'SSMOTE-AVERAGE' ,dataset_dir = dataset_dir )
    print_results("BSMOTE", sum_metrics_BSMOTE , filename = 'BSMOTE-AVERAGE' ,dataset_dir = dataset_dir )
    print_results("ADASYN", sum_metrics_ADASYN , filename = 'ADASYN-AVERAGE' ,dataset_dir = dataset_dir )
    print_results("MWMOTE", sum_metrics_MWMOTE , filename = 'MWMOTE-AVERAGE' ,dataset_dir = dataset_dir )
    print_results("ASUWO", sum_metrics_ASUWO , filename = 'ASUWO-AVERAGE' ,dataset_dir = dataset_dir )
    print_results("KMSMOTE", sum_metrics_KMSMOTE , filename = 'KMSMOTE-AVERAGE' ,dataset_dir = dataset_dir )
    print_results("GSMOTE", sum_metrics_GSMOTE , filename = 'GSMOTE-AVERAGE' ,dataset_dir = dataset_dir )
    

# SADCO

In [8]:
def GENERATEWITHSADCO(data_set_name ,dim, data_allmajority, data_allminority, IMratio, majlabel = +1 , minlabel = -1  ):
    #***************************************************************** pre processing and ESSP **********************************************************
    

   
    
    
    
    
    minENNdist,minenemykneigor,majENNdist,majenemykneigor,majmindistmax = KNNenemyPoint(data_allminority[:, :-1] , data_allmajority[: ,:-1])  
    
    #*********************
    majlabel = majlabel
    minlabel = minlabel
    mindradious = []
    #**********************
    
    for i in range(0,data_allminority.shape[0]):
        ilabel = minlabel
        dradius = calradious(i,ilabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor) #labels of in our dataset
        mindradious.append([i, *data_allminority[i, :-1], dradius])
    
    
    
    
    data = data_allminority[:, :-1]
    
    radiouses = np.array(mindradious, dtype=object)
    
    #*************************************************************************************************************************************************
    
    # ***************************************** merge sphere and calculate number of sphere and all samples belong to sphere****************************
    
    sorted_dataradious, pddist = unionSphere(data,radiouses)
    
    
    uncoveredpoint = np.unique(sorted_dataradious[:,-1]) # NAME OF REMAINING SPHERES   ابر کره های منحصر به فرد انتخاب می شوند
    
    numofsphere = []                                     # NUMBER OF SAMPLES IN EACH SPHERE  تعداد نمونه های داخل هر ابر کره
    
    for j in uncoveredpoint:
         numofsphere = np.append(numofsphere,np.sum(sorted_dataradious[:,-1]==j) )
    
    uncoveredpoint = np.array(uncoveredpoint, dtype=int)
    
    #****************************************************************************************************************************************
    
    # *************************************************************************** Compute complexity measure ************************************
    
     
    complexitymeasure = len(uncoveredpoint) / data_allminority.shape[0]
    
    # Extract final minimum centers and radii
    finalmincenters = sorted_dataradious[uncoveredpoint, 1:-2]  # Fix indexing issue  # مرکز ابر کره های منحصر به فرد ویژگی هارا گرفته و در اینجا ذخیره می کنیم
    finalminradius = sorted_dataradious[uncoveredpoint, -2]  # Fix indexing issue      # شعاع ابر کره های حول این مراکز
    
    # Set complexity measure and compute oversample size
    complexitymeasure = 1
    
    oversamplesize = np.ceil(complexitymeasure * (data_allmajority.shape[0] - data_allminority.shape[0]))
    # **********************************************************************************************************************************************
    
    
    
    
    # ******************************************************************************** OVERSAMPLING SIZE OF EACH SPHERE *********************************
    
    
    # Define alpha and beta
    alpha = 1
    beta = 1
    
    # Convert finalminradius and numofsphere properly
    finalminradius = np.asarray(finalminradius, dtype=float).reshape(-1)
    numofsphere = np.asarray(numofsphere, dtype=float).reshape(-1)
    
    # Compute pi and pj
    pi = np.exp(-alpha * finalminradius)  
    pj = np.exp(-beta * numofsphere)
    
    # Normalize pi
    pi = (pi + pj) / np.sum(pi + pj)
    
    # Compute sub-oversample sizes
    suboversamplesizes = np.ceil(pi * oversamplesize)
    
    #***********************************************************************************************************************************************
    
    #******************************************************************************************GENERATE NEW SAMPLES ********************************
    
    
    
    generatedX = []  
    for i in range(len(uncoveredpoint)):
        X = gnerandsphere(finalmincenters[i, :], dim, int(suboversamplesizes[i]), finalminradius[i])
        generatedX.append(X)
    
    generatedX = np.vstack(generatedX)  # Convert to NumPy array
    
    
    
    #***********************************************************************************************************************************************
    
    
    
    
    
    #*******************************************************************************table ***********************************************************
    
    #print(" \n************************************************ table ******************************************************************************* \n")
    #print( "mindradious is \n{0}\n ".format(mindradious))
    #print("UNIQUE HYPER SPERE \n{0}\n".format(uncoveredpoint))
    #print("TABLE OF RESULTS IS \n{0}\n".format(sorted_dataradious))
    #print("final min centers is \n{0}\n".format(finalmincenters))
    #print("final min radius is \n{0}\n".format(finalminradius))
    #print("shape final min centers is \n{0}\n".format(finalmincenters.shape))
    #print("shape final min radius is \n{0}\n".format(finalminradius.shape))
    

    
    #print("number of samples in each sphere \n{0}\n".format(numofsphere))
   # print("generated samples is \n{0}\n".format(generatedX))
    #print("total number of syntic samples is \n{0}\n".format((len(generatedX))))
    #print(" \n************************************************ table ******************************************************************************** \n")

    return  data_allminority , data_allmajority , mindradious , uncoveredpoint , sorted_dataradious , finalmincenters , finalminradius , numofsphere , generatedX

# SADCOHC1

In [10]:
def generate_samples(space_point ,d , density ,r , degree ):


        s = space_point + np.random.uniform(-1, 1, (density, d)) * r * degree

        return s


#*********************************************************************method for calculating  distance of majority samples from line *************************************** 
def point_to_line_distance(P, A, B):

    P, A, B = np.asarray(P, dtype=float), np.asarray(A, dtype=float), np.asarray(B, dtype=float)
    d = B - A
    v = P - A
    d_norm_sq = np.dot(d, d)
    
    if d_norm_sq == 0:
        return np.linalg.norm(v)
    
    proj_d_v = np.dot(v, d) / d_norm_sq * d
    perp_vec = v - proj_d_v
    return np.linalg.norm(perp_vec)
    


#*********************************************************************method for calculating  distance of majority samples from line *************************************** 




def cylinder1(data_allminority, data_allmajority, finalmincenters, finalminradius,generatedX, th  , density , degree ,keep=3, th_end=np.inf , limit = np.inf , use_enemy=True): 

    
    
    #************************************************************************* genete cylinders in space **********************************************************

    counter = 0 
    
    space_dim = finalmincenters.shape[1]
    dim = finalmincenters.shape[0]
    
    from_center1_to_center2 = {}
    from_sample_to_line = {}

    new_finalmincenters = []
    new_finalminradius = []



    for i, center1 in enumerate(finalmincenters):
        for j, center2 in enumerate(finalmincenters):
            if j > i:
                diff = center2 - center1
                from_center1_to_center2[(i, j)] = {"center1": center1, "center2": center2, "diff": diff,"index":-1}
    
                max_distance1, max_distance2 = -np.inf, -np.inf
                max_sample1, max_sample2 = None, None
                max_index1, max_index2 = None, None  # To store the indices of the maximum samples
                if use_enemy ==True : 
                    for idx, p in enumerate(data_allmajority[:, :-1]):  # Assuming last column is a label or not needed
                        distance = np.linalg.norm(p - center1) + np.linalg.norm(p - center2)  # Combined distance
        
                        if distance >= max_distance1:
                            max_distance2, max_sample2, max_index2 = max_distance1, max_sample1, max_index1
                            max_distance1, max_sample1, max_index1 = distance, p, idx
                        elif distance > max_distance2:
                            max_distance2, max_sample2, max_index2 = distance, p, idx
        
                    # Store the max samples, distances, and their indices
                    from_center1_to_center2[(i, max_index1)] = {"center1": center1, "center2": max_sample1, "diff": max_distance1,"index":max_index1}
                    from_center1_to_center2[(i, max_index2)] = {"center1": center1, "center2": max_sample2, "diff": max_distance2,"index":max_index2}                               
                    from_center1_to_center2[(j, max_index1)] = {"center1": center2, "center2": max_sample1, "diff": max_distance1,"index":max_index1}
                    from_center1_to_center2[(j, max_index2)] = {"center1": center2, "center2": max_sample2, "diff": max_distance2,"index":max_index2}      




    

       
    for idx, line_data in from_center1_to_center2.items():
        A, B = line_data["center1"], line_data["center2"]
        min_distance = min(
            point_to_line_distance(P, A, B) 
            for idx2, P in enumerate(data_allmajority[:, :-1]) if from_center1_to_center2[idx]["index"] != idx2
        )
        
        mid  = (A + B) / 2
        mid_left = (A+mid) / 2
        mid_right = (B+mid)/2

        from_sample_to_line[idx] = {
                'mid': mid,
                'mid_left': mid_left,
                'mid_right': mid_right,
                'rad': min_distance
        }
        
        if   keep==3:
            if min_distance>th and min_distance<th_end:
                new_finalmincenters.extend([mid, mid_left, mid_right])
                new_finalminradius.extend([min_distance] * 3)
                counter+=3
        if   keep==2:
            if min_distance>th and min_distance<th_end:
                new_finalmincenters.extend([mid, mid_left])
                new_finalminradius.extend([min_distance] *2)
                counter+=2


        if   keep==1:
            if min_distance>th and min_distance<th_end:
                new_finalmincenters.extend([mid])
                new_finalminradius.extend([min_distance])
                counter+=1

        if counter>=limit:
            break
   

            

    # Convert to NumPy arrays
    new_finalmincenters = np.array(new_finalmincenters, dtype=float)
    new_finalminradius = np.array(new_finalminradius, dtype=float).reshape(-1)

    #************************************************************************* genete cylinders in space **********************************************************


    

    #*************************************************************************** generate new samples ***************************************************
    
    new_generatedX = []


    for i in range(len(new_finalmincenters)):
        X = generate_samples(new_finalmincenters[i, :], space_dim , density , new_finalminradius[i] , degree)
        new_generatedX.append(X)
    
    new_generatedX = np.vstack(new_generatedX)  


  

     #*************************************************************************** generate new samples ***************************************************

        #*************************************************************************table ****************************************************************************


    #print("Distance from sample to line:\n", from_sample_to_line, "\n")
    #print("Line from center1 to center2:\n", from_center1_to_center2, "\n")
    #print("Length from center1 to center2 (total lines) is {0}\n".format(len(from_center1_to_center2)))
    #print("Length from sample to line (total lines) is {0}\n".format(len(from_sample_to_line)))
    #print("Final min centers shape:", new_finalmincenters.shape)
    #print("Final min radiuses shape:", new_finalminradius.shape)
    #print("Total number of new samples {0}\n".format(len(new_generatedX)))


    #*************************************************************************table ****************************************************************************
    
    return finalmincenters , finalminradius , new_finalmincenters , new_finalminradius , new_generatedX , from_center1_to_center2 , from_sample_to_line

# SADOHC2

In [12]:
import numpy as np

def sample_hyper_cylinder(N, d, A, B, r):
   
    N = int(N)
    d = int(d)
    A, B = np.asarray(A, dtype=float), np.asarray(B, dtype=float)
    
    # Compute the axis direction (unit vector)
    axis_direction = B - A
    axis_direction /= np.linalg.norm(axis_direction)
    
    # Random positions along the cylinder's axis
    t = np.random.rand(N, 1)
    axis_positions = A + t * (B - A)
    
    # Generate random perpendicular displacement vectors
    random_directions = np.random.randn(N, d)
    random_directions -= (random_directions @ axis_direction)[:, None] * axis_direction
    random_directions /= np.linalg.norm(random_directions, axis=1, keepdims=True)
    
    # Random radii (uniform in area)
    random_radii = r * np.sqrt(np.random.rand(N, 1))
    displacement = random_directions * random_radii
    
    return axis_positions + displacement


def generate_samples2(density, space_dim, A, B, r):
 
    return sample_hyper_cylinder(density, space_dim, A, B, r)

def cylinder2(data_allminority, data_allmajority, finalmincenters, finalminradius, generatedX, th, density, th_end=np.inf, limit=np.inf, use_enemy=False):
   
    counter = 0
    space_dim = finalmincenters.shape[1]
    from_center1_to_center2 = {}
    left_axis, right_axis, radius = [], [], []
    
    for i, center1 in enumerate(finalmincenters):
        for j, center2 in enumerate(finalmincenters):
            if j > i:
                diff = center2 - center1
                from_center1_to_center2[(i, j)] = {"center1": center1, "center2": center2, "diff": diff, "index":-1}
                
                
                if use_enemy ==True : 
                    max_distance1, max_distance2 = -np.inf, -np.inf
                    max_sample1, max_sample2 = None, None
                    max_index1, max_index2 = None, None  # To store the indices of the maximum samples
                    for idx, p in enumerate(data_allmajority[:, :-1]):  # Assuming last column is a label or not needed
                        distance = np.linalg.norm(p - center1) + np.linalg.norm(p - center2)  # Combined distance
                        
                        if distance > max_distance1:
                            max_distance2, max_sample2, max_index2 = max_distance1, max_sample1, max_index1
                            max_distance1, max_sample1, max_index1 = distance, p, idx
                        elif distance > max_distance2:
                            max_distance2, max_sample2, max_index2 = distance, p, idx
                    # Store the max samples, distances, and their indices
                    from_center1_to_center2[(i, max_index1)] = {"center1": center1, "center2": max_sample1, "diff": max_distance1,"index":max_index1}
                    from_center1_to_center2[(i, max_index2)] = {"center1": center1, "center2": max_sample2, "diff": max_distance2,"index":max_index2}                               
                    from_center1_to_center2[(j, max_index1)] = {"center1": center2, "center2": max_sample1, "diff": max_distance1,"index":max_index1}
                    from_center1_to_center2[(j, max_index2)] = {"center1": center2, "center2": max_sample2, "diff": max_distance2,"index":max_index2} 
    
    for idx, line_data in from_center1_to_center2.items():
        A, B = line_data["center1"], line_data["center2"]
        min_distance = min(
            point_to_line_distance(P, A, B) 
            for idx2, P in enumerate(data_allmajority[:, :-1]) if from_center1_to_center2[idx]["index"] != idx2
        )
        
        if th < min_distance < th_end:
            left_axis.append(A)
            right_axis.append(B)
            radius.append(min_distance)
            counter += 1
            
        if counter >= limit:
            break
    
    new_generatedX = np.vstack([generate_samples2(density, space_dim, left_axis[i], right_axis[i], radius[i]) for i in range(len(left_axis))])

    new_generatedX = np.array(new_generatedX)

    
    return finalmincenters, finalminradius, left_axis, right_axis, radius, new_generatedX, from_center1_to_center2

# glass-0-1-2-3_vs_4-5-6  D30 ok

In [100]:
printResults("glass-0-1-2-3_vs_4-5-6.txt",th=1 ,density=10, degree=0.4 , keep=1  ,th_end=np.inf, limit = 10 , use_enemy=False , th2=0.2 ,density2=10, th_end2=1, limit2 = 10 , use_enemy2=False, n_splits=5 , times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.9817518939393939
  F1-score: 0.8987035364838568
  G-Mean: 0.9505673499952987
  Recall Class -1.0: 0.9606060606060607
  Recall Class 1.0: 0.9428030303030303
  Precision Class -1.0: 0.8537819587819588
  Precision Class 1.0: 0.9878672876775343

XGBoost:
  AUC: 0.9739835858585858
  F1-score: 0.9000688083845978
  G-Mean: 0.9472201701444276
  Recall Class -1.0: 0.9472727272727274
  Recall Class 1.0: 0.9489267676767678
  Precision Class -1.0: 0.8657128057128057
  Precision Class 1.0: 0.9835255509828565

SVM:
  AUC: 0.9828977272727273
  F1-score: 0.9014869188782233
  G-Mean: 0.9520242740250457
  Recall Class -1.0: 0.9606060606060607
  Recall Class 1.0: 0.9446969696969698
  Precision Class -1.0: 0.8551637251637252
  Precision Class 1.0: 0.9874877810361682

SADCOC2 Results:
Random Forest:
  AUC: 0.9808143939393938
  F1-score: 0.8939340369775153
  G-Mean: 0.953165165438269
  Recall Class -1.0: 0.9733333333333333
  Recall Class 1.0: 0.9347222222222221
  Pre

# glass-0-1-6_vs_5  D29 ok

In [262]:
printResults("glass-0-1-6_vs_5.txt",th=0 ,density=3, degree=0.7 , keep=3   ,th_end=np.inf, limit = np.inf,use_enemy=False,th2=0,density2=30, th_end2=np.inf, limit2 =np.inf,use_enemy2=False, n_splits=5 , times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.9890476190476192
  F1-score: 0.7755555555555557
  G-Mean: 0.9311360008092525
  Recall Class -1.0: 0.9
  Recall Class 1.0: 0.979047619047619
  Precision Class -1.0: 0.7333333333333333
  Precision Class 1.0: 0.9943915343915344

XGBoost:
  AUC: 0.9742857142857144
  F1-score: 0.6247619047619047
  G-Mean: 0.8206988689332237
  Recall Class -1.0: 0.7666666666666667
  Recall Class 1.0: 0.9619047619047619
  Precision Class -1.0: 0.548888888888889
  Precision Class 1.0: 0.9881582633053222

SVM:
  AUC: 0.9838095238095238
  F1-score: 0.6866666666666668
  G-Mean: 0.8358816001074504
  Recall Class -1.0: 0.8333333333333334
  Recall Class 1.0: 0.9752380952380952
  Precision Class -1.0: 0.6666666666666666
  Precision Class 1.0: 0.9907879307879308

SADCOC2 Results:
Random Forest:
  AUC: 0.9909523809523809
  F1-score: 0.6933333333333334
  G-Mean: 0.8195284146916916
  Recall Class -1.0: 0.8
  Recall Class 1.0: 0.980952380952381
  Precision Class -1.0: 0.7
  Precisi

# glass-0-1-6_vs_2 D28 ok

In [275]:
printResults("glass-0-1-6_vs_2.txt",th=0.2 ,density=7, degree=0.1 , keep=3   ,th_end=0.5, limit = np.inf,use_enemy=False,th2=0 ,density2=7 , th_end2=0.5, limit2 = np.inf,use_enemy2=False, n_splits=5 , times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.8180952380952379
  F1-score: 0.3381481481481482
  G-Mean: 0.5839701997105075
  Recall Class -1.0: 0.4222222222222222
  Recall Class 1.0: 0.8876190476190478
  Precision Class -1.0: 0.3317460317460318
  Precision Class 1.0: 0.9416739235411342

XGBoost:
  AUC: 0.8666666666666667
  F1-score: 0.4450793650793651
  G-Mean: 0.5805962551419519
  Recall Class -1.0: 0.38333333333333325
  Recall Class 1.0: 0.9676190476190476
  Precision Class -1.0: 0.5966666666666666
  Precision Class 1.0: 0.9426889303359892

SVM:
  AUC: 0.9000000000000001
  F1-score: 0.46478188478188487
  G-Mean: 0.833052959201441
  Recall Class -1.0: 0.8833333333333334
  Recall Class 1.0: 0.8114285714285716
  Precision Class -1.0: 0.32360028860028855
  Precision Class 1.0: 0.9871960989202369

SADCOC2 Results:
Random Forest:
  AUC: 0.8367460317460318
  F1-score: 0.3836507936507937
  G-Mean: 0.5504373996920934
  Recall Class -1.0: 0.3833333333333334
  Recall Class 1.0: 0.9409523809523811
  

# Liver D27 ok

In [280]:
printResults("Liver.txt",th=1,density=3, degree=0.0005 , keep=3   , th_end=np.inf,limit=5 , use_enemy=False,th2=1,density2=3, th_end2=np.inf,limit2=5 , use_enemy2=False,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.7573850574712644
  F1-score: 0.6495759343033504
  G-Mean: 0.6822501209133579
  Recall Class -1.0: 0.703448275862069
  Recall Class 1.0: 0.6633333333333334
  Precision Class -1.0: 0.6047679789544117
  Precision Class 1.0: 0.7544915351223402

XGBoost:
  AUC: 0.7650574712643677
  F1-score: 0.6563504195188019
  G-Mean: 0.698405256916021
  Recall Class -1.0: 0.6689655172413792
  Recall Class 1.0: 0.7316666666666667
  Precision Class -1.0: 0.6473038838602482
  Precision Class 1.0: 0.7532280494840795

SVM:
  AUC: 0.7329885057471265
  F1-score: 0.6372684104124817
  G-Mean: 0.6426299539353774
  Recall Class -1.0: 0.7586206896551725
  Recall Class 1.0: 0.5483333333333333
  Precision Class -1.0: 0.5510661410222564
  Precision Class 1.0: 0.758699137346521

SADCOC2 Results:
Random Forest:
  AUC: 0.7622988505747127
  F1-score: 0.6524792522026014
  G-Mean: 0.683846408341465
  Recall Class -1.0: 0.7080459770114942
  Recall Class 1.0: 0.6633333333333334
  Precis

# Pima D26 ok

In [283]:
printResults("Pima.txt",th=1,density=5, degree=0.5, keep=1   , th_end=np.inf, limit=100 , use_enemy=False,th2=1,density2=5, th_end2=np.inf, limit2=60 , use_enemy2=False,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.8163275098998369
  F1-score: 0.6737027344351432
  G-Mean: 0.7460557859464682
  Recall Class -1.0: 0.7513859771721407
  Recall Class 1.0: 0.7419999999999999
  Precision Class -1.0: 0.61239388930712
  Precision Class 1.0: 0.8477470955893518

XGBoost:
  AUC: 0.8056151875145584
  F1-score: 0.6611126637609744
  G-Mean: 0.7361720374539494
  Recall Class -1.0: 0.7102725366876309
  Recall Class 1.0: 0.7646666666666665
  Precision Class -1.0: 0.6207247866902686
  Precision Class 1.0: 0.8315916584512656

SVM:
  AUC: 0.8160111809923133
  F1-score: 0.6776718767373089
  G-Mean: 0.7373106262133429
  Recall Class -1.0: 0.8667598416026088
  Recall Class 1.0: 0.6286666666666666
  Precision Class -1.0: 0.5573578791278048
  Precision Class 1.0: 0.8988819970608894

SADCOC2 Results:
Random Forest:
  AUC: 0.8223604705334266
  F1-score: 0.6826828089793759
  G-Mean: 0.7537531398813658
  Recall Class -1.0: 0.7550430934078732
  Recall Class 1.0: 0.7539999999999999
  Prec

# Wine D25 ok

In [141]:
printResults("Wine.txt",th=2 ,density=5 ,degree=0.05, keep=3 , th_end=5 , limit=40, use_enemy=False,th2=0 ,density2=5 , th_end2=np.inf , limit2=np.inf, use_enemy2=False,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 1.0
  F1-score: 0.96210222526012
  G-Mean: 0.9843860548424725
  Recall Class -1.0: 1.0
  Recall Class 1.0: 0.9692307692307693
  Precision Class -1.0: 0.9290909090909093
  Precision Class 1.0: 1.0

XGBoost:
  AUC: 0.9989743589743589
  F1-score: 0.9504540666554397
  G-Mean: 0.9770133962730474
  Recall Class -1.0: 0.9933333333333334
  Recall Class 1.0: 0.9615384615384617
  Precision Class -1.0: 0.9148873348873351
  Precision Class 1.0: 0.9974358974358974

SVM:
  AUC: 0.9997435897435897
  F1-score: 0.9674158658678782
  G-Mean: 0.9823568592038202
  Recall Class -1.0: 0.985925925925926
  Recall Class 1.0: 0.9794871794871796
  Precision Class -1.0: 0.952929292929293
  Precision Class 1.0: 0.9950617283950617

SADCOC2 Results:
Random Forest:
  AUC: 0.9998717948717949
  F1-score: 0.9528214475582897
  G-Mean: 0.9804498661738142
  Recall Class -1.0: 1.0
  Recall Class 1.0: 0.9615384615384617
  Precision Class -1.0: 0.9123232323232325
  Precision Class 1.0: 1.

# Heberman D24 ok

In [143]:
printResults("Heberman.txt",th=0.2,density=3, degree=0.05 , keep=1 , th_end=np.inf , limit=100, use_enemy=False,th2=0.2,density2=3 , th_end2=np.inf , limit2=100, use_enemy2=False,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.6871623093681918
  F1-score: 0.4722237524446388
  G-Mean: 0.6255901544944138
  Recall Class -1.0: 0.5637254901960784
  Recall Class 1.0: 0.714074074074074
  Precision Class -1.0: 0.41607636515482593
  Precision Class 1.0: 0.8232940891882877

XGBoost:
  AUC: 0.6786982570806103
  F1-score: 0.42077670362335046
  G-Mean: 0.5791178733324817
  Recall Class -1.0: 0.4607843137254902
  Recall Class 1.0: 0.7466666666666665
  Precision Class -1.0: 0.3930845647300755
  Precision Class 1.0: 0.7954334680732198

SVM:
  AUC: 0.7043627450980393
  F1-score: 0.48846211656120553
  G-Mean: 0.634148858955563
  Recall Class -1.0: 0.6995098039215686
  Recall Class 1.0: 0.5866666666666667
  Precision Class -1.0: 0.37855264206500727
  Precision Class 1.0: 0.8493568004331311

SADCOC2 Results:
Random Forest:
  AUC: 0.6910729847494552
  F1-score: 0.45359852623010516
  G-Mean: 0.610107624237071
  Recall Class -1.0: 0.551470588235294
  Recall Class 1.0: 0.6933333333333332
  P

# SPECT D23 ok

In [286]:
printResults("SPECT.txt",th=0.1 ,density=10, degree=0.25, keep=3   , th_end=np.inf,limit=100 , use_enemy=False, th2=0.1 ,density2=8, th_end2=np.inf,limit2=100 , use_enemy2=False,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.746870417888563
  F1-score: 0.6136351925564119
  G-Mean: 0.6692959017080667
  Recall Class -1.0: 0.593939393939394
  Recall Class 1.0: 0.7638440860215054
  Precision Class -1.0: 0.6456220476506386
  Precision Class 1.0: 0.7305856146723019

XGBoost:
  AUC: 0.6949184384164224
  F1-score: 0.5826334848270566
  G-Mean: 0.6417335754493064
  Recall Class -1.0: 0.5727272727272726
  Recall Class 1.0: 0.7261424731182795
  Precision Class -1.0: 0.60161574730606
  Precision Class 1.0: 0.7087520152810288

SVM:
  AUC: 0.7475088587487781
  F1-score: 0.6704376989693802
  G-Mean: 0.6718759867357668
  Recall Class -1.0: 0.827272727272727
  Recall Class 1.0: 0.5538978494623658
  Precision Class -1.0: 0.567374392534952
  Precision Class 1.0: 0.8321818792108153

SADCOC2 Results:
Random Forest:
  AUC: 0.7455186950146628
  F1-score: 0.6399921328262992
  G-Mean: 0.6901238657272486
  Recall Class -1.0: 0.6363636363636364
  Recall Class 1.0: 0.7553763440860215
  Precisio

# abalone9-18 D22 ok

In [291]:
printResults("abalone9-18.txt",th=1 ,density=200, degree=0.4 , keep=1   , th_end=np.inf ,limit=50, use_enemy=False,th2=1 ,density2=200, th_end2=np.inf ,limit2=50, use_enemy2=True,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.8327340477841643
  F1-score: 0.3716908547249105
  G-Mean: 0.608614084546066
  Recall Class -1.0: 0.40277777777777773
  Recall Class 1.0: 0.953080150922106
  Precision Class -1.0: 0.3687021475256769
  Precision Class 1.0: 0.9634128252694545

XGBoost:
  AUC: 0.804796708667834
  F1-score: 0.3803801885535632
  G-Mean: 0.581107607931349
  Recall Class -1.0: 0.35555555555555557
  Recall Class 1.0: 0.9680701012024401
  Precision Class -1.0: 0.44439153439153434
  Precision Class 1.0: 0.9611343399886334

SVM:
  AUC: 0.8801306953308962
  F1-score: 0.33348544464598473
  G-Mean: 0.7777637781500604
  Recall Class -1.0: 0.7379629629629628
  Recall Class 1.0: 0.8330864981134737
  Precision Class -1.0: 0.21798599080782985
  Precision Class 1.0: 0.9815129904626143

SADCOC2 Results:
Random Forest:
  AUC: 0.8334827323091631
  F1-score: 0.37844773473503224
  G-Mean: 0.6776906629533214
  Recall Class -1.0: 0.5064814814814815
  Recall Class 1.0: 0.9269544060086745
  

## Ecoli D21 ok

In [149]:
printResults("ECOLI.txt",th=1 ,density=3, degree=1 , keep=3  , th_end=np.inf ,limit=np.inf, use_enemy=True,th2=1 ,density2=9, th_end2=np.inf ,limit2=np.inf, use_enemy2=True,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.9618409660514924
  F1-score: 0.8326574451059578
  G-Mean: 0.8955806231883728
  Recall Class -1.0: 0.8296969696969697
  Recall Class 1.0: 0.9707184628237261
  Precision Class -1.0: 0.8471813371813373
  Precision Class 1.0: 0.9687944412951335

XGBoost:
  AUC: 0.9540229361281993
  F1-score: 0.8162431771127424
  G-Mean: 0.8808704586875717
  Recall Class -1.0: 0.8024242424242426
  Recall Class 1.0: 0.9707184628237261
  Precision Class -1.0: 0.8373905723905725
  Precision Class 1.0: 0.9641488411001471

SVM:
  AUC: 0.9622826004404952
  F1-score: 0.6874016408407823
  G-Mean: 0.8858820628938513
  Recall Class -1.0: 0.9303030303030302
  Recall Class 1.0: 0.8472848788638261
  Precision Class -1.0: 0.5602881561163296
  Precision Class 1.0: 0.985562841482107

SADCOC2 Results:
Random Forest:
  AUC: 0.9644245082402978
  F1-score: 0.8300033681040547
  G-Mean: 0.9004032819726366
  Recall Class -1.0: 0.8412121212121213
  Recall Class 1.0: 0.9660401002506268
  Pre

# Page Block D20 ok

In [151]:
printResults("PageBlock.txt",th=0 ,density=10, degree=0.05, keep=3   , th_end=np.inf,limit=10, use_enemy=False,th2=0 ,density2=10, th_end2=np.inf,limit2=10, use_enemy2=False,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.9905823432821155
  F1-score: 0.8622007520653064
  G-Mean: 0.9541827335305059
  Recall Class -1.0: 0.9357142857142858
  Recall Class 1.0: 0.9731329409189071
  Precision Class -1.0: 0.8000454954171504
  Precision Class 1.0: 0.9925287914502723

XGBoost:
  AUC: 0.9898372054585601
  F1-score: 0.875046856860317
  G-Mean: 0.9451251937329676
  Recall Class -1.0: 0.911309523809524
  Recall Class 1.0: 0.9803247191391468
  Precision Class -1.0: 0.8424808728708759
  Precision Class 1.0: 0.9897957178192165

SVM:
  AUC: 0.9772308633467622
  F1-score: 0.695743902432053
  G-Mean: 0.9410163463499442
  Recall Class -1.0: 0.9791666666666669
  Recall Class 1.0: 0.9044018511573877
  Precision Class -1.0: 0.5400424370963265
  Precision Class 1.0: 0.9973770613492638

SADCOC2 Results:
Random Forest:
  AUC: 0.9902997022138452
  F1-score: 0.8630431433288492
  G-Mean: 0.9548150657290062
  Recall Class -1.0: 0.9369047619047619
  Recall Class 1.0: 0.9732002770796686
  Preci

# Page Block 1 D19 ok

In [153]:
printResults("PageBlock1.txt",th=0 ,density=10, degree=0.005, keep=1 ,th_end=np.inf,limit=10, use_enemy=False,th2=0 ,density2=10,th_end2=np.inf,limit2=10, use_enemy2=False,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.9884492761209986
  F1-score: 0.8776414308865985
  G-Mean: 0.9574086110685679
  Recall Class -1.0: 0.928003108003108
  Recall Class 1.0: 0.9879471336431551
  Precision Class -1.0: 0.8344475751441359
  Precision Class 1.0: 0.995368877496319

XGBoost:
  AUC: 0.9891663049445151
  F1-score: 0.8893825459551323
  G-Mean: 0.951457771566697
  Recall Class -1.0: 0.9138150738150738
  Recall Class 1.0: 0.9909279405666286
  Precision Class -1.0: 0.8676281717762406
  Precision Class 1.0: 0.9944757606031337

SVM:
  AUC: 0.9821488508006809
  F1-score: 0.6345588332578871
  G-Mean: 0.9477468079846745
  Recall Class -1.0: 0.9655011655011656
  Recall Class 1.0: 0.9304688671837086
  Precision Class -1.0: 0.4737667694909549
  Precision Class 1.0: 0.9976427447370175

SADCOC2 Results:
Random Forest:
  AUC: 0.989003200595752
  F1-score: 0.8781466823166812
  G-Mean: 0.9584464153581693
  Recall Class -1.0: 0.9300543900543902
  Recall Class 1.0: 0.9878821567537521
  Precis

# Car f D18 ok

In [155]:
printResults("Car.txt",th=0.3 ,density=5, degree=0.0005, keep=1  , th_end=np.inf,limit=20, use_enemy=False,th2=0 ,density2=5, th_end2=np.inf,limit2=100, use_enemy2=False,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.9940817584604211
  F1-score: 0.7515665694736159
  G-Mean: 0.8244311089367877
  Recall Class -1.0: 0.6864468864468863
  Recall Class 1.0: 0.9943756900714642
  Precision Class -1.0: 0.8487781662781664
  Precision Class 1.0: 0.987056549246214

XGBoost:
  AUC: 0.9979912945497336
  F1-score: 0.8648775755753881
  G-Mean: 0.9191707499222476
  Recall Class -1.0: 0.8509157509157511
  Recall Class 1.0: 0.9951789029228698
  Precision Class -1.0: 0.8900912159735691
  Precision Class 1.0: 0.993795982189205

SVM:
  AUC: 0.9763645481964376
  F1-score: 0.4939077639117465
  G-Mean: 0.9475099126092037
  Recall Class -1.0: 0.980952380952381
  Recall Class 1.0: 0.9162213809922469
  Precision Class -1.0: 0.3315812918205182
  Precision Class 1.0: 0.9991444780898681

SADCOC2 Results:
Random Forest:
  AUC: 0.9934102733432983
  F1-score: 0.7442386437298982
  G-Mean: 0.820289781454922
  Recall Class -1.0: 0.6816849816849817
  Recall Class 1.0: 0.9941742802023806
  Precis

# poker-8-9_vs_6  D17 ok

In [294]:
printResults("poker-8-9_vs_6.txt",th=1.2,density=50, degree=0.05, keep=3  , th_end=1.7,limit=100, use_enemy=False,th2=1.2 ,density2=50, th_end2=1.7,limit2=100, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 1.0
  F1-score: 1.0
  G-Mean: 1.0
  Recall Class -1.0: 1.0
  Recall Class 1.0: 1.0
  Precision Class -1.0: 1.0
  Precision Class 1.0: 1.0

XGBoost:
  AUC: 1.0
  F1-score: 1.0
  G-Mean: 1.0
  Recall Class -1.0: 1.0
  Recall Class 1.0: 1.0
  Precision Class -1.0: 1.0
  Precision Class 1.0: 1.0

SVM:
  AUC: 0.9934703196347034
  F1-score: 0.6999111999112001
  G-Mean: 0.9235555570601178
  Recall Class -1.0: 0.8666666666666667
  Recall Class 1.0: 0.989041095890411
  Precision Class -1.0: 0.6006084656084655
  Precision Class 1.0: 0.9977018625253201

SADCOC2 Results:
Random Forest:
  AUC: 1.0
  F1-score: 0.9925925925925926
  G-Mean: 0.9929618127333277
  Recall Class -1.0: 0.9866666666666667
  Recall Class 1.0: 1.0
  Precision Class -1.0: 1.0
  Precision Class 1.0: 0.9997724687144482

XGBoost:
  AUC: 1.0
  F1-score: 1.0
  G-Mean: 1.0
  Recall Class -1.0: 1.0
  Recall Class 1.0: 1.0
  Precision Class -1.0: 1.0
  Precision Class 1.0: 1.0

SVM:
  AUC: 0.98114

# poker-8_vs_6  D16  OK

In [382]:
printResults("poker-8_vs_6.txt",th=0.5 ,density=7, degree=0.05, keep=3  , th_end=np.inf,limit=20, use_enemy=False,th2=0.5 ,density2=5,th_end2=np.inf,limit2=20, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.9985350076103501
  F1-score: 0.8187301587301586
  G-Mean: 0.8310665067152153
  Recall Class -1.0: 0.7555555555555554
  Recall Class 1.0: 1.0
  Precision Class -1.0: 1.0
  Precision Class 1.0: 0.9970544131928822

XGBoost:
  AUC: 1.0
  F1-score: 0.9238095238095239
  G-Mean: 0.9347150628109128
  Recall Class -1.0: 0.8944444444444444
  Recall Class 1.0: 1.0
  Precision Class -1.0: 1.0
  Precision Class 1.0: 0.9988654392359901

SVM:
  AUC: 0.9899543378995435
  F1-score: 0.5088291664762253
  G-Mean: 0.9225851914210332
  Recall Class -1.0: 0.8777777777777778
  Recall Class 1.0: 0.9812785388127857
  Precision Class -1.0: 0.36975283975283973
  Precision Class 1.0: 0.9986182813231568

SADCOC2 Results:
Random Forest:
  AUC: 0.9978405631659055
  F1-score: 0.8387301587301587
  G-Mean: 0.8606903138708268
  Recall Class -1.0: 0.7722222222222223
  Recall Class 1.0: 1.0
  Precision Class -1.0: 1.0
  Precision Class 1.0: 0.9972788958239626

XGBoost:
  AUC: 1.0
  

# poker-9_vs_7 f D15 OK

In [367]:
printResults("poker-9_vs_7.txt",th=2 ,density=3, degree=0.1, keep=3  , th_end=np.inf,limit=np.inf, use_enemy=False,th2=2 ,density2=10,th_end2=np.inf,limit2=np.inf, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.9522458628841608
  F1-score: 0.4222222222222222
  G-Mean: 0.43570226039551585
  Recall Class -1.0: 0.36666666666666664
  Recall Class 1.0: 0.9986111111111112
  Precision Class -1.0: 0.9333333333333333
  Precision Class 1.0: 0.9793934240362812

XGBoost:
  AUC: 0.9564125295508276
  F1-score: 0.42
  G-Mean: 0.4728244331596937
  Recall Class -1.0: 0.43333333333333335
  Recall Class 1.0: 0.9859633569739954
  Precision Class -1.0: 0.6444444444444444
  Precision Class 1.0: 0.9805778694456506

SVM:
  AUC: 0.9843971631205674
  F1-score: 0.6977777777777779
  G-Mean: 0.9120167320501922
  Recall Class -1.0: 0.8666666666666667
  Recall Class 1.0: 0.9787825059101657
  Precision Class -1.0: 0.6555555555555554
  Precision Class 1.0: 0.9943223010244288

SADCOC2 Results:
Random Forest:
  AUC: 0.9773936170212766
  F1-score: 0.48888888888888893
  G-Mean: 0.548098246646462
  Recall Class -1.0: 0.4666666666666667
  Recall Class 1.0: 0.995774231678487
  Precision Clas

# vehicle0 D14 ok


In [303]:
 printResults("vehicle0.txt",th=0,density=40, degree=1, keep=3 , th_end=np.inf,limit=10 ,use_enemy=False,th2=0 ,density2=40, th_end2=np.inf,limit2=10, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.9948288419491462
  F1-score: 0.9110319034475738
  G-Mean: 0.9649066607889745
  Recall Class -1.0: 0.9866666666666666
  Recall Class 1.0: 0.9438958457563109
  Precision Class -1.0: 0.8485123504846169
  Precision Class 1.0: 0.9956825356376883

XGBoost:
  AUC: 0.9950451508340596
  F1-score: 0.9317902033976391
  G-Mean: 0.9675654293290622
  Recall Class -1.0: 0.9698717948717949
  Recall Class 1.0: 0.9654979129397734
  Precision Class -1.0: 0.8977665541168958
  Precision Class 1.0: 0.9905685776020421

SVM:
  AUC: 0.9886608641805421
  F1-score: 0.8776339820827049
  G-Mean: 0.9553476266548633
  Recall Class -1.0: 1.0
  Recall Class 1.0: 0.9129516994633274
  Precision Class -1.0: 0.7840818240588002
  Precision Class 1.0: 1.0

SADCOC2 Results:
Random Forest:
  AUC: 0.9957469802608442
  F1-score: 0.922254615094494
  G-Mean: 0.9721839468197805
  Recall Class -1.0: 0.9966666666666666
  Recall Class 1.0: 0.9485311071357584
  Precision Class -1.0: 0.860152308

# vehicle1 D13 ok

In [199]:
printResults("vehicle1.txt",th=0.1 ,density=5, degree=0.05, keep=1 , th_end=np.inf,limit=100, use_enemy=False,th2=0 ,density2=11,th_end2=np.inf,limit2=5, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.8622515375236305
  F1-score: 0.6364838980030871
  G-Mean: 0.7688640098450199
  Recall Class -1.0: 0.7446793516560958
  Recall Class 1.0: 0.7954243386243387
  Precision Class -1.0: 0.5565733098950877
  Precision Class 1.0: 0.9009127568167777

XGBoost:
  AUC: 0.8645088806112061
  F1-score: 0.6117156951873896
  G-Mean: 0.7323809757327905
  Recall Class -1.0: 0.6324524312896406
  Recall Class 1.0: 0.8510857142857146
  Precision Class -1.0: 0.5943037916129571
  Precision Class 1.0: 0.870932258500052

SVM:
  AUC: 0.8928162801883732
  F1-score: 0.686685117888801
  G-Mean: 0.8253179277775831
  Recall Class -1.0: 0.9490838618745596
  Recall Class 1.0: 0.7185862433862432
  Precision Class -1.0: 0.538596074161808
  Precision Class 1.0: 0.9768494758714231

SADCOC2 Results:
Random Forest:
  AUC: 0.8605861024419162
  F1-score: 0.6329532366668841
  G-Mean: 0.7654865207363437
  Recall Class -1.0: 0.7385835095137419
  Recall Class 1.0: 0.7948825396825399
  Preci

# vehicle2 D12 OK

In [318]:
printResults("vehicle2.txt",th=0 ,density=20, degree=0.2, keep=3 , th_end=np.inf,limit=100, use_enemy=False,th2=0 ,density2=20, th_end2=np.inf,limit2=100, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.9979281083257828
  F1-score: 0.9490025768464079
  G-Mean: 0.9768110972147759
  Recall Class -1.0: 0.9862226920366456
  Recall Class 1.0: 0.9676063492063491
  Precision Class -1.0: 0.9153933019549123
  Precision Class 1.0: 0.9950854656851678

XGBoost:
  AUC: 0.9971153506269785
  F1-score: 0.9647497272305882
  G-Mean: 0.9846834715670881
  Recall Class -1.0: 0.9923890063424948
  Recall Class 1.0: 0.9771640211640212
  Precision Class -1.0: 0.9395907898264244
  Precision Class 1.0: 0.9973331268550781

SVM:
  AUC: 0.9938465340000222
  F1-score: 0.9296324913458162
  G-Mean: 0.9704440853417423
  Recall Class -1.0: 0.9908386187455954
  Recall Class 1.0: 0.9506582010582011
  Precision Class -1.0: 0.8769433441187199
  Precision Class 1.0: 0.9967026521514967

SADCOC2 Results:
Random Forest:
  AUC: 0.9981058849849549
  F1-score: 0.9373831452669715
  G-Mean: 0.9748623148430835
  Recall Class -1.0: 0.9954545454545456
  Recall Class 1.0: 0.9548698412698413
  Pr

# vehicle3 D11  ok

In [203]:
printResults("vehicle3.txt",th=1 ,density=30, degree=0.1, keep=3 , th_end=np.inf,limit=50, use_enemy=False,th2=0 ,density2=20, th_end2=np.inf,limit2=5, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.8596333888496496
  F1-score: 0.6160343096485771
  G-Mean: 0.7526966392872931
  Recall Class -1.0: 0.7091177556293835
  Recall Class 1.0: 0.8006832479273427
  Precision Class -1.0: 0.5462587811786273
  Precision Class 1.0: 0.8917890763882409

XGBoost:
  AUC: 0.8697356628871004
  F1-score: 0.6208940388868031
  G-Mean: 0.7443172486843285
  Recall Class -1.0: 0.6557401255075673
  Recall Class 1.0: 0.846960796567096
  Precision Class -1.0: 0.5923129109354796
  Precision Class 1.0: 0.880556597002915

SVM:
  AUC: 0.8763347411030986
  F1-score: 0.6676236208141556
  G-Mean: 0.8138450917891474
  Recall Class -1.0: 0.932595053525286
  Recall Class 1.0: 0.710827813190018
  Precision Class -1.0: 0.5206873724516118
  Precision Class 1.0: 0.9692376288554303

SADCOC2 Results:
Random Forest:
  AUC: 0.8646252551764363
  F1-score: 0.6357504971072421
  G-Mean: 0.7676141129696507
  Recall Class -1.0: 0.726504245108896
  Recall Class 1.0: 0.8117401991417742
  Precisi

# new-thyroid2 D10 ok

In [312]:
printResults("new-thyroid2.txt",th=0 ,density=5, degree=0.005, keep=3 , th_end=np.inf,limit=np.inf, use_enemy=False,th2=0 ,density2=5, th_end2=np.inf,limit2=np.inf, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.9976190476190477
  F1-score: 0.9241086691086693
  G-Mean: 0.9721386770950612
  Recall Class -1.0: 0.9714285714285715
  Recall Class 1.0: 0.974074074074074
  Precision Class -1.0: 0.8876984126984127
  Precision Class 1.0: 0.9945445445445446

XGBoost:
  AUC: 0.9973544973544974
  F1-score: 0.9279181929181931
  G-Mean: 0.969127412936759
  Recall Class -1.0: 0.961904761904762
  Recall Class 1.0: 0.9777777777777777
  Precision Class -1.0: 0.9031746031746032
  Precision Class 1.0: 0.9926926926926927

SVM:
  AUC: 0.9989417989417989
  F1-score: 0.9075778813549712
  G-Mean: 0.9673086252219849
  Recall Class -1.0: 0.9714285714285715
  Recall Class 1.0: 0.9648148148148147
  Precision Class -1.0: 0.8630687830687832
  Precision Class 1.0: 0.9945445445445446

SADCOC2 Results:
Random Forest:
  AUC: 0.9981481481481482
  F1-score: 0.936886446886447
  G-Mean: 0.9749494362290065
  Recall Class -1.0: 0.9714285714285715
  Recall Class 1.0: 0.9796296296296296
  Precis

# paw02a-600-5-50-BI D9 ok

In [207]:
printResults("paw02a-600-5-50-BI.txt",th=0.01 ,density=5, degree=1, keep=3 , th_end=np.inf,limit=100, use_enemy=False,th2=0.01 ,density2=9, th_end2=np.inf,limit2=100, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.9180166666666666
  F1-score: 0.6354323734785722
  G-Mean: 0.8431899597928803
  Recall Class -1.0: 0.8600000000000001
  Recall Class 1.0: 0.828
  Precision Class -1.0: 0.5062224200946362
  Precision Class 1.0: 0.9672705680287801

XGBoost:
  AUC: 0.9192333333333332
  F1-score: 0.6320607618770232
  G-Mean: 0.8299878465187945
  Recall Class -1.0: 0.8200000000000001
  Recall Class 1.0: 0.8433333333333335
  Precision Class -1.0: 0.5183959458360963
  Precision Class 1.0: 0.9594636571893288

SVM:
  AUC: 0.9344000000000001
  F1-score: 0.5973593895355085
  G-Mean: 0.8474476497733929
  Recall Class -1.0: 0.9633333333333333
  Recall Class 1.0: 0.746
  Precision Class -1.0: 0.43359991612901905
  Precision Class 1.0: 0.9902424117072163

SADCOC2 Results:
Random Forest:
  AUC: 0.9209666666666667
  F1-score: 0.6292826612001982
  G-Mean: 0.8461544895348222
  Recall Class -1.0: 0.8800000000000001
  Recall Class 1.0: 0.8146666666666668
  Precision Class -1.0: 0.491

# paw02a-600-5-30-BI D8  OK

In [209]:
printResults("paw02a-600-5-30-BI.txt", th=0.01 ,density=5, degree=0.05, keep=3 , th_end=np.inf,limit=50, use_enemy=False, th2=0.01 ,density2=5,th_end2=np.inf,limit2=50, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.9288000000000001
  F1-score: 0.6682425420384791
  G-Mean: 0.8498362877261323
  Recall Class -1.0: 0.8366666666666667
  Recall Class 1.0: 0.8646666666666667
  Precision Class -1.0: 0.5596958014927982
  Precision Class 1.0: 0.9637507203862812

XGBoost:
  AUC: 0.9336166666666668
  F1-score: 0.679786845560308
  G-Mean: 0.857105453360072
  Recall Class -1.0: 0.8466666666666666
  Recall Class 1.0: 0.8693333333333334
  Precision Class -1.0: 0.5712383324672033
  Precision Class 1.0: 0.9660635533229681

SVM:
  AUC: 0.9404333333333333
  F1-score: 0.6044747709164269
  G-Mean: 0.8520084083473386
  Recall Class -1.0: 0.9699999999999999
  Recall Class 1.0: 0.7493333333333332
  Precision Class -1.0: 0.4404035324568722
  Precision Class 1.0: 0.9921688714162396

SADCOC2 Results:
Random Forest:
  AUC: 0.9333000000000001
  F1-score: 0.6643025175349885
  G-Mean: 0.8566209825159542
  Recall Class -1.0: 0.8633333333333332
  Recall Class 1.0: 0.8513333333333333
  Prec

# 04clover5z-600-5-70-BI D7  OK

In [211]:
printResults("04clover5z-600-5-70-BI.txt",th=0.01 ,density=4, degree=0.5, keep=3 , th_end=np.inf,limit=100, use_enemy=False,th2=0.01 ,density2=5,th_end2=np.inf,limit2=100, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.8733000000000001
  F1-score: 0.5828151464371264
  G-Mean: 0.81078184003773
  Recall Class -1.0: 0.8300000000000001
  Recall Class 1.0: 0.7953333333333333
  Precision Class -1.0: 0.4518346975043536
  Precision Class 1.0: 0.9596148921392292

XGBoost:
  AUC: 0.8741666666666668
  F1-score: 0.5893087271667456
  G-Mean: 0.8102541406854998
  Recall Class -1.0: 0.8166666666666665
  Recall Class 1.0: 0.8079999999999998
  Precision Class -1.0: 0.46450458479870244
  Precision Class 1.0: 0.9572834866973636

SVM:
  AUC: 0.8575499999999999
  F1-score: 0.4918202203411078
  G-Mean: 0.7625963341642392
  Recall Class -1.0: 0.9366666666666668
  Recall Class 1.0: 0.6226666666666666
  Precision Class -1.0: 0.3342463122609262
  Precision Class 1.0: 0.9803048788684791

SADCOC2 Results:
Random Forest:
  AUC: 0.8683
  F1-score: 0.572726115325608
  G-Mean: 0.8159457786219697
  Recall Class -1.0: 0.8733333333333334
  Recall Class 1.0: 0.7633333333333333
  Precision Class 

# 04clover5z-600-5-60-BI D6  OK

In [213]:
printResults("04clover5z-600-5-60-BI.txt",th=0 ,density=2, degree=0.005, keep=3 , th_end=np.inf,limit=40, use_enemy=False,th2=0 ,density2=5,th_end2=np.inf,limit2=40, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.8757833333333334
  F1-score: 0.57140492325481
  G-Mean: 0.7885194341868661
  Recall Class -1.0: 0.7666666666666668
  Recall Class 1.0: 0.8166666666666665
  Precision Class -1.0: 0.45950004619691226
  Precision Class 1.0: 0.9467849479889464

XGBoost:
  AUC: 0.8750166666666667
  F1-score: 0.5673076649223944
  G-Mean: 0.7837033882001821
  Recall Class -1.0: 0.7533333333333333
  Recall Class 1.0: 0.8193333333333334
  Precision Class -1.0: 0.4579145303176922
  Precision Class 1.0: 0.9437543402722908

SVM:
  AUC: 0.8587333333333335
  F1-score: 0.5057624360684847
  G-Mean: 0.7747992864253792
  Recall Class -1.0: 0.9199999999999998
  Recall Class 1.0: 0.6553333333333332
  Precision Class -1.0: 0.34975836380031744
  Precision Class 1.0: 0.9771692886070724

SADCOC2 Results:
Random Forest:
  AUC: 0.8809166666666666
  F1-score: 0.5650358339275319
  G-Mean: 0.7921173914686218
  Recall Class -1.0: 0.7900000000000001
  Recall Class 1.0: 0.8000000000000002
  Pr

# 03subcl5-600-5-70-BI D5 OK

In [215]:
printResults("03subcl5-600-5-70-BI.txt",th=0 ,density=3, degree=0.005, keep=3 , th_end=np.inf,limit=40, use_enemy=False,th2=0 ,density2=3, th_end2=np.inf,limit2=40, use_enemy2=False,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.8600166666666667
  F1-score: 0.5159717546301883
  G-Mean: 0.7496476525764569
  Recall Class -1.0: 0.7166666666666667
  Recall Class 1.0: 0.7873333333333332
  Precision Class -1.0: 0.4049591205918904
  Precision Class 1.0: 0.9332899081549305

XGBoost:
  AUC: 0.85015
  F1-score: 0.5274881967250146
  G-Mean: 0.74994819884845
  Recall Class -1.0: 0.7033333333333334
  Recall Class 1.0: 0.8073333333333333
  Precision Class -1.0: 0.42629517247307996
  Precision Class 1.0: 0.9324344957557921

SVM:
  AUC: 0.8454166666666668
  F1-score: 0.5057020069562522
  G-Mean: 0.772086364540175
  Recall Class -1.0: 0.8966666666666666
  Recall Class 1.0: 0.6673333333333333
  Precision Class -1.0: 0.35324249466585805
  Precision Class 1.0: 0.9702260186536161

SADCOC2 Results:
Random Forest:
  AUC: 0.8608833333333334
  F1-score: 0.5419665316106187
  G-Mean: 0.7744422783577378
  Recall Class -1.0: 0.7633333333333334
  Recall Class 1.0: 0.7886666666666666
  Precision Clas

# 03subcl5-600-5-60-B D4  OK

In [332]:
printResults("03subcl5-600-5-60-BI.txt",th=0 ,density=5, degree=0.005, keep=3 , th_end=np.inf,limit=20, use_enemy=False,th2=0 ,density2=5, th_end2=np.inf,limit2=20, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.8733833333333334
  F1-score: 0.5591471708015355
  G-Mean: 0.7841613794587993
  Recall Class -1.0: 0.7699999999999999
  Recall Class 1.0: 0.8019999999999998
  Precision Class -1.0: 0.44238338992069154
  Precision Class 1.0: 0.9463966228288705

XGBoost:
  AUC: 0.8666666666666667
  F1-score: 0.5516550011108922
  G-Mean: 0.770405779819579
  Recall Class -1.0: 0.73
  Recall Class 1.0: 0.8153333333333334
  Precision Class -1.0: 0.44588190729967525
  Precision Class 1.0: 0.9380907167015163

SVM:
  AUC: 0.8598
  F1-score: 0.5243484138372583
  G-Mean: 0.790354003902684
  Recall Class -1.0: 0.9199999999999999
  Recall Class 1.0: 0.6806666666666666
  Precision Class -1.0: 0.3676161781429199
  Precision Class 1.0: 0.9774884892947905

SADCOC2 Results:
Random Forest:
  AUC: 0.8695666666666667
  F1-score: 0.5558063360002042
  G-Mean: 0.7868403706081833
  Recall Class -1.0: 0.7866666666666666
  Recall Class 1.0: 0.7900000000000001
  Precision Class -1.0: 0.4326

# zoo-3  D3 OK

In [74]:
printResults("zoo-3.txt",th=0.1 ,density=40, degree=0.1, keep=3 , th_end=np.inf,limit=np.inf, use_enemy=False,th2=0.3 ,density2=60, th_end2=np.inf,limit2=10, use_enemy2=False,n_splits=5 ,times=3) 

SADCOC1 Results:
Random Forest:
  AUC: 0.9410526315789475
  F1-score: 0.6
  G-Mean: 0.6
  Recall Class -1.0: 0.6
  Recall Class 1.0: 0.9966666666666666
  Precision Class -1.0: 0.9333333333333333
  Precision Class 1.0: 0.98

XGBoost:
  AUC: 0.7964912280701756
  F1-score: 0.16666666666666666
  G-Mean: 0.19657888653670094
  Recall Class -1.0: 0.2
  Recall Class 1.0: 0.9659649122807018
  Precision Class -1.0: 0.5555555555555555
  Precision Class 1.0: 0.9589278752436646

SVM:
  AUC: 0.9796491228070175
  F1-score: 0.5333333333333333
  G-Mean: 0.5947557658700032
  Recall Class -1.0: 0.6
  Recall Class 1.0: 0.9796491228070175
  Precision Class -1.0: 0.8333333333333334
  Precision Class 1.0: 0.9796296296296295

SADCOC2 Results:
Random Forest:
  AUC: 0.9343859649122808
  F1-score: 0.5555555555555555
  G-Mean: 0.5965338640843647
  Recall Class -1.0: 0.6
  Recall Class 1.0: 0.9864912280701755
  Precision Class -1.0: 0.8666666666666667
  Precision Class 1.0: 0.9798245614035087

XGBoost:
  AUC: 0.78

# abalone-21_vs_8  D2 OK

In [345]:
printResults("abalone-21_vs_8.txt",th=0 ,density=10, degree=0.001, keep=1 , th_end=np.inf,limit=np.inf, use_enemy=False,th2=0 ,density2=10, th_end2=np.inf,limit2=np.inf, use_enemy2=False,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.9232598457796409
  F1-score: 0.4953339253339254
  G-Mean: 0.7843390131627263
  Recall Class -1.0: 0.6555555555555556
  Recall Class 1.0: 0.9758888371370907
  Precision Class -1.0: 0.4238888888888888
  Precision Class 1.0: 0.9911130048472852

XGBoost:
  AUC: 0.8679363107867998
  F1-score: 0.5314959114959115
  G-Mean: 0.7789072730130407
  Recall Class -1.0: 0.6777777777777777
  Recall Class 1.0: 0.9782331936034777
  Precision Class -1.0: 0.4881746031746032
  Precision Class 1.0: 0.991717081395941

SVM:
  AUC: 0.9542807362555848
  F1-score: 0.39149402025563007
  G-Mean: 0.8629807758717839
  Recall Class -1.0: 0.8111111111111112
  Recall Class 1.0: 0.9412099570460072
  Precision Class -1.0: 0.272535705035705
  Precision Class 1.0: 0.9951830941597795

SADCOC2 Results:
Random Forest:
  AUC: 0.9359308423467716
  F1-score: 0.467989047989048
  G-Mean: 0.8235711213744444
  Recall Class -1.0: 0.7222222222222221
  Recall Class 1.0: 0.9658903896910419
  Prec

# cleveland-0_vs_4  D1 OK

In [352]:
printResults("cleveland-0_vs_4.txt",th=0 ,density=30, degree=0.1, keep=3 , th_end=np.inf,limit=np.inf, use_enemy=False,th2=1 ,density2=30, th_end2=3,limit2=np.inf, use_enemy2=False,n_splits=5 ,times=3)

SADCOC1 Results:
Random Forest:
  AUC: 0.9777777777777777
  F1-score: 0.7133333333333333
  G-Mean: 0.7589095106928472
  Recall Class -1.0: 0.6444444444444444
  Recall Class 1.0: 0.9958333333333333
  Precision Class -1.0: 0.9555555555555555
  Precision Class 1.0: 0.9703366649690179

XGBoost:
  AUC: 0.9840277777777778
  F1-score: 0.7598412698412698
  G-Mean: 0.9000181393234822
  Recall Class -1.0: 0.8555555555555555
  Recall Class 1.0: 0.9666666666666667
  Precision Class -1.0: 0.7288888888888889
  Precision Class 1.0: 0.9876262626262625

SVM:
  AUC: 0.9826388888888888
  F1-score: 0.722063492063492
  G-Mean: 0.9040172417014581
  Recall Class -1.0: 0.8666666666666668
  Recall Class 1.0: 0.9604166666666667
  Precision Class -1.0: 0.6566666666666667
  Precision Class 1.0: 0.9876106894370652

SADCOC2 Results:
Random Forest:
  AUC: 0.9802083333333333
  F1-score: 0.6688888888888889
  G-Mean: 0.7442774194171488
  Recall Class -1.0: 0.6222222222222222
  Recall Class 1.0: 0.9895833333333334
  Pre